In [ ]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from util import run_perm_test_mean, plot_pdf_cdf

import warnings
warnings.filterwarnings("ignore")

# set defaults
plt.style.use('seaborn-white')   # seaborn custom plot style
plt.rc('figure', dpi=100, figsize=(7, 5))   # set default size/resolution
plt.rc('font', size=12)   # font size

# Lecture 18

### Permutation Tests

## Today's lecture

- So far, we've been assessing models given a single random sample.
    - We flip a coin 400 times. Are the flips consistent with the coin being fair?
    - Did the jury panel in the Swain case look like a random sample from the eligible population?
    - Are the test scores for the TA's section a random sample from the class's scores?
- But we often have *two* random samples we wish to compare.
    - Outcomes of patients assigned to control group and treatment group in a pharmaceutical study.
    - Number of clicks from people who saw version A of an advertisement vs. version B
- **A/B testing**: decide whether two random samples come from the same distribution.
- The "exciting" outcome is typically that which *rejects* the null hypothesis.

# Example 1: Birth Weight and Smoking

## Smoking and birth weight

- Is there a significant difference in the weight of babies born to mothers who smoke, vs. non-smokers?
- Two groups:
    - babies whose mothers smoke
    - babies whose mothers do not smoke

In [ ]:
baby = pd.read_csv('baby.csv')
baby.head()

In [ ]:
#: we only need "Birth Weight" and "Maternal Smoker"
smoking_and_birthweight = baby[['Maternal Smoker', 'Birth Weight']]
smoking_and_birthweight.head()

## First, some exploratory analysis
* How many are in each group?
* What is the average weight within each group?

In [ ]:
# how many are in each group?
smoking_and_birthweight.groupby('Maternal Smoker').count()

In [ ]:
# what is the average weight within each group?
smoking_and_birthweight.groupby('Maternal Smoker').mean()

## Visualizing the distribution of each group
- Does the difference we see reflect a real difference in the population?
- Or is it just due to random chance?

In [ ]:
title='Birth Weight by whether mother smoked'
smoking_and_birthweight.groupby('Maternal Smoker')['Birth Weight'].plot(kind='kde', legend=True, subplots=False, title=title);

## The question:

Try a hypothesis test?
* But *both* samples are fixed beforehand?
* But what is the (null) distribution being simulated?

In [ ]:
title='Birth Weight by whether mother smoked'
smoking_and_birthweight.groupby('Maternal Smoker')['Birth Weight'].plot(kind='kde', legend=True, subplots=False, title=title);

## Testing the hypothesis

- **Null hypothesis**: In the population, birth weights of smokers and non-smokers have the same distribution.
    - I.e., what we saw is due to random chance.
- **Alternative hypothesis**: In the population, babies born to smokers typically have lower birth weight.

## Discussion question

- What is a reasonable statistic to compute in order to test the null hypothesis?

## Answer: difference between means

In [ ]:
means_table = smoking_and_birthweight.groupby('Maternal Smoker').mean()
means_table

In [ ]:
observed_difference = means_table.diff().iloc[-1,0]
observed_difference

## Testing through simulation

- **Statistic**: Difference between means.
- **Null hypothesis**: The two groups are sampled from the same distribution.
- Note that the null hypothesis doesn't say *what* the distribution is.
    - Different from jury panel example, fair coin example, etc.
    - We can't draw directly from the distribution!
- We have to do something a bit more clever.

## Implications of the null hypothesis

- Under the null hypothesis, both groups are sampled from the same distribution.
- If true, then the group label (`Maternal Smoker`) has no effect on the birth weight.
- We saw one assignment of group labels:

In [ ]:
smoking_and_birthweight.head()

- But (under the null hypothesis) we were just as likely to see *any other* assignment.

## Permutation tests

- Perhaps the difference in means we saw is due to random chance in assignment.
- **Permutation test**: Shuffle the group labels a bunch of times; how often do we see a statistic this extreme?
- Randomly permuting labels is equivalent to randomly assigning birth weights to groups (without changing group sizes)
- If we *rarely* see something this extreme, then the null hypothesis doesn't look likely.

## Permutation tests with dataframes

- We want to randomly shuffle the `Maternal Smoker` column.
- To shuffle rows, we can use `.sample(replace=False, frac=1)`.
- Notice: Both rows are shuffled together (i.e., not independently)!

In [ ]:
# (re)run multiple times!
smoking_and_birthweight.head().sample(replace=False, frac=1)

## Discussion Question

In the birthweight example, we want to shuffle only one column. For the purpose of permutation testing, 
does it matter which column we shuffle -- the `Maternal Smoker` column or the `Birth Weight` column?

- A) Yes
- B) No

## Shuffling the groups

In [ ]:
smoking_and_birthweight.head()

In [ ]:
#: it doesn't matter which column we shuffle, but it will be more convenient to shuffle weights
shuffled_weights = (
    smoking_and_birthweight['Birth Weight']
    .sample(replace=False, frac=1)
    .reset_index(drop=True)
)

original_and_shuffled = (
    smoking_and_birthweight
    .assign(**{'Shuffled Birth Weight': shuffled_weights})
)

original_and_shuffled.head(10)

## How close are the means of the shuffled groups?

In [ ]:
#: shuffling the weights makes it easier to do this...
original_and_shuffled.groupby('Maternal Smoker').mean()

In [ ]:
#: the distribution of the shuffled groups

fig, axes = plt.subplots(1,2, figsize=(18,8))

title = 'shuffled birth weights by maternal smoker'
original_and_shuffled.groupby('Maternal Smoker')['Shuffled Birth Weight'].plot(kind='kde', title=title, ax=axes[0])

title = 'birth weights by maternal smoker'
original_and_shuffled.groupby('Maternal Smoker')['Birth Weight'].plot(kind='kde', title=title, ax=axes[1]);

## Simulation

- This was just one random shuffle.
- How likely is it that a random shuffle results in a 9+ ounce difference in means?
- We have to repeat the shuffling a bunch of times. On each iteration:
    1. Shuffle the weights.
    2. Put them in a table.
    3. Compute difference in group means.

In [ ]:
n_repetitions = 500

differences = []
for _ in range(n_repetitions):
    
    # shuffle the weights
    shuffled_weights = (
        smoking_and_birthweight['Birth Weight']
        .sample(replace=False, frac=1)
        .reset_index(drop=True)
    )
    
    # put them in a table
    shuffled = (
        smoking_and_birthweight
        .assign(**{'Shuffled Birth Weight': shuffled_weights})
    )
    
    # compute the group differences
    group_means = (
        shuffled
        .groupby('Maternal Smoker')
        .mean()
        .loc[:, 'Shuffled Birth Weight']
    )
    difference = group_means.diff().iloc[-1]
    
    # add it to the list of results
    differences.append(difference)

## Conclusion of the test

In [ ]:
pd.Series(differences).plot(kind='hist', density=True)
plt.scatter(observed_difference, 0, color='red', s=40);

- Under the null hypothesis, we rarely see differences as large as this.
- Therefore, we reject the null hypothesis: the two groups do not come from the same distribution.

## Caution!

- We **cannot** conclude that smoking *causes* lower birth weight!
- This was an observational study; there may be confounding factors.
    - Maybe smokers are more likely to drink caffeine, and caffeine causes lower birth weight.
- But it suggests that it is worth studying with a randomized controlled trial.

## A slightly different example

- Is there a difference in birthweight between babies born to:
    - Group A: Mothers over 25
    - Group B: Mothers 25 or under

In [ ]:
over_25 = baby['Maternal Age'] > 25
age_and_birthweight = (
    baby[['Birth Weight']]
    .assign(**{'Over 25': over_25})
)

age_and_birthweight.head()

In [ ]:
#: the group means
group_means = age_and_birthweight.groupby('Over 25').mean()
group_means

In [ ]:
#: the difference between them
observed_difference = group_means['Birth Weight'].diff().iloc[-1]
observed_difference

## The permutation test

- **Null hypothesis**: Birth weights for both groups come from the same distribution.
- **Alternative hypothesis**: No, mothers below 25 have heavier babies.
- We run a permutation test with the difference in means as the statistic.

## Simulation

In [ ]:
n_repetitions = 500

differences = []
for _ in range(n_repetitions):
    
    # shuffle the weights
    shuffled_weights = (
        age_and_birthweight['Birth Weight']
        .sample(replace=False, frac=1)
        .reset_index(drop=True)
    )
    
    # put them in a table
    shuffled = (
        age_and_birthweight
        .assign(**{'Shuffled Birth Weight': shuffled_weights})
    )
    
    # compute the group differences
    group_means = (
        shuffled
        .groupby('Over 25')
        .mean()
        .loc[:, 'Shuffled Birth Weight']
    )
    difference = group_means.diff().iloc[-1]
    
    # add it to the list of results
    differences.append(difference)

## Conclusion of the test

- Do we reject the null hypothesis?
- What can we compute to tell us the degree of uncertainty here?

In [ ]:
#: visualize
pd.Series(differences).plot(kind='hist', density=True, alpha=0.8)
plt.scatter(observed_difference, 0, color='red', s=40);

## The p-value

- The probability of seeing a difference of means at least as extreme as the observed, under the null hypothesis.

In [ ]:
np.count_nonzero(differences >= observed_difference) / n_repetitions

## Example: Assessing Missingness

* Data on ticketed cars: VIN, Make, Year, Color
* Is car color missing at random, dependent on car year?
    - Are the distributions of year dissimilar when color is null vs not null?
    - How different is different enough?
    
Use a permutation test!

In [ ]:
cars = pd.read_csv('cars.csv')
cars.head()

In [ ]:
# proportion of car color missing
cars.car_color.isnull().mean()

In [ ]:
cars['car_color_isnull'] = cars.car_color.isnull()

In [ ]:
(
    cars
    .pivot_table(index='car_year', columns='car_color_isnull', values=None, aggfunc='size')
    .fillna(0)
    .apply(lambda x:x/x.sum())
    .plot(title='distribution of car years by color=missing/not missing')
);

### Assessing missingness of car color on year

* "Are the two distributions (missing/not missing) of car year generated from the same distribution?"
* Reduction: If so, then their means should be similar!
* Run a permutation test.

In [ ]:
observed_difference, differences = run_perm_test_mean(cars, 'car_color_isnull', 'car_year')

In [ ]:
#: visualize: differences in means 
pd.Series(differences).plot(kind='hist', density=True, alpha=0.8)
plt.scatter(observed_difference, 0, color='red', s=40);

### Example: assessing missingness of car make on color

* "Are the two distributions (missing/not missing) of car make generated from the same distribution?"
* Car make is categorical. How to measure similarity without means?
    - use total variation distance

In [ ]:
cars['car_make_isnull'] = cars.car_make.isnull()

In [ ]:
cars.head()

In [ ]:
emp_distributions = (
    cars
    .pivot_table(columns='car_make_isnull', index='car_color', values=None, aggfunc='size')
    .fillna(0)
    .apply(lambda x:x/x.sum())
)

emp_distributions.plot(kind='bar', title='distribution of car colors');

In [ ]:
observed_tvd = np.sum(np.abs(emp_distributions.diff(axis=1).iloc[:,-1]))
observed_tvd

In [ ]:
n_repetitions = 500

car_make_color = cars.copy()[['car_color', 'car_make_isnull']]
tvds = []
for _ in range(n_repetitions):
    
    # shuffle the colors
    shuffled_colors = (
        car_make_color['car_color']
        .sample(replace=False, frac=1)
        .reset_index(drop=True)
    )
    
    # put them in a table
    shuffled = (
        car_make_color
        .assign(**{'Shuffled Color': shuffled_colors})
    )
    
    # compute the tvd
    shuffed_emp_distributions = (
        shuffled
        .pivot_table(columns='car_make_isnull', index='Shuffled Color', values=None, aggfunc='size')
        .fillna(0)
        .apply(lambda x:x/x.sum())
    )
    
    tvd = np.sum(np.abs(shuffed_emp_distributions.diff(axis=1).iloc[:,-1]))
    # add it to the list of results
    
    tvds.append(tvd)

In [ ]:
#: visualize
pd.Series(tvds).plot(kind='hist', density=True, alpha=0.8)
plt.scatter(observed_tvd, 0, color='red', s=40);

### Example: assessing missingness in payments data

* Payment information for purchases: credit card type, credit card number, date of birth.
* Is the credit card number missing at random dependent on the type of card?

In [ ]:
payments = pd.read_csv('payment.csv')
payments['cc_isnull'] = payments.credit_card_number.isnull()

In [ ]:
payments.head()

In [ ]:
emp_distributions = (
    payments
    .pivot_table(columns='cc_isnull', index='credit_card_type', aggfunc='size')
    .fillna(0)
    .apply(lambda x:x/x.sum())
)

emp_distributions.plot(kind='bar', title='distribution of card types');

In [ ]:
observed_tvd = np.sum(np.abs(emp_distributions.diff(axis=1).iloc[:,-1]))
observed_tvd

In [ ]:
n_repetitions = 500

payments_type = payments.copy()[['credit_card_type', 'cc_isnull']]
tvds = []
for _ in range(n_repetitions):
    
    # shuffle the colors
    shuffled_types = (
        payments_type['credit_card_type']
        .sample(replace=False, frac=1)
        .reset_index(drop=True)
    )
    
    # put them in a table
    shuffled = (
        payments_type
        .assign(**{'Shuffled Types': shuffled_types})
    )
    
    # compute the tvd
    shuffed_emp_distributions = (
        shuffled
        .pivot_table(columns='cc_isnull', index='Shuffled Types', values=None, aggfunc='size')
        .fillna(0)
        .apply(lambda x:x/x.sum())
    )
    
    tvd = np.sum(np.abs(shuffed_emp_distributions.diff(axis=1).iloc[:,-1]))
    # add it to the list of results
    
    tvds.append(tvd)

### Example: assessing missingness in payments data

* Is the credit card number missing at random dependent on the type of card?
* As always, set significance level **beforehand**:
    - How important is the column in the modeling process?
    - How many null values are there?
* Consideration: how important is a faithful imputation?

In [ ]:
#: visualize
pd.Series(tvds).plot(kind='hist', density=True, alpha=0.8)
plt.scatter(observed_tvd, 0, color='red', s=40);

In [ ]:
# p-value
np.count_nonzero(tvds <= observed_tvd) / len(tvds)

### Example: assessing missingness in payments data

* Is the credit card number missing at random dependent on the age of shopper?
* For quantitative distributions, we've compared means of two groups.

In [ ]:
payments['date_of_birth'] = pd.to_datetime(payments.date_of_birth)
payments['age'] = (2019 - payments.date_of_birth.dt.year)

In [ ]:
# are the distributions similar?
# Where are the differences? Are they noise, or real?
payments.groupby('cc_isnull').age.plot(kind='kde', title='distribution of ages by missingness of CC', legend=True);

In [ ]:
# Run a permutation test
observed_difference, differences = run_perm_test_mean(payments, 'cc_isnull', 'age')

In [ ]:
#: visualize
pd.Series(differences).plot(kind='hist', density=True, alpha=0.8)
plt.scatter(observed_difference, 0, color='red', s=40);

In [ ]:
# p-value
np.count_nonzero(differences <= observed_difference) / len(differences)

### Example: assessing missingness in payments data

* Is the credit card number missing at random dependent on the age of shopper?
* The two distributions "look different", but have similar means.
    - Means may be too coarse a statistic
    - Need a different metric for quantitative distributions

## KS-Statistic

<div class="image-txt-container">
    
* Kolmogorov-Smirnov Test: similarity between two distributions.
* KS-statistics measures the area between two empirical distributions.
* Like TVD, for quantitative distributions.
* Uses *Cumulative Distribution Function* instead of density.
    
    
<img src="KS2_Example.png">

</div>

### Using KS as the similarity measure in permutation tests

* Is the credit card number missing at random, dependent on the age of shopper?
* Measure similarity of missing vs not missing distributions using KS-statistic.

In [ ]:
payments.groupby('cc_isnull').age.plot(kind='kde', title='distribution of ages by missingness of CC', legend=True);

In [ ]:
plot_pdf_cdf(payments);

In [ ]:
# import KS from scipy
from scipy.stats import ks_2samp

observed_ks, _ = ks_2samp(
    payments.loc[payments['cc_isnull'], 'age'],
    payments.loc[~payments['cc_isnull'], 'age']
)

In [ ]:
observed_ks

In [ ]:
n_repetitions = 500

kslist = []
for _ in range(n_repetitions):
    
    # shuffle the ages
    shuffled_age = (
        payments['age']
        .sample(replace=False, frac=1)
        .reset_index(drop=True)
    )
    
    # 
    shuffled = (
        payments
        .assign(**{'Shuffled Age': shuffled_age})
    )

    ks, _ = ks_2samp(
        shuffled.loc[shuffled['cc_isnull'], 'Shuffled Age'],
        shuffled.loc[~shuffled['cc_isnull'], 'Shuffled Age']
    )
    
    # add it to the list of results
    kslist.append(ks)

In [ ]:
pd.Series(kslist).plot(kind='hist', density=True, alpha=0.8)
plt.scatter(observed_ks, 0, color='red', s=40);

In [ ]:
# p-value
np.count_nonzero(kslist >= observed_ks) / len(kslist)

## Example: Deflategate

## Did the New England Patriots cheat?

<div class="image-txt-container">
    
<img width="50%" src="./deflate.jpg">

- On January 18, 2015, the Patriots played the Indianapolis Colts for a spot in the Super Bowl
- The Patriots won, 45 -- 7. They went on to win the Super Bowl
- After the game, it was alleged that the Patriots intentionally deflated footballs (making them easier to catch)

</div>

## Background

- Each team brings 12 footballs to the game.
- NFL rules stipulate: each ball must be inflated to between 12.5 and 13.5 pounds per square inch (psi).
- Before the game, officials found that all of the Patriot's balls were at about 12.5 psi, all of the Colts were about 13.0 psi.
- In the second quarter, Colts intercepted a Patriots ball and notified officials that it felt under-inflated.
- At halftime, two officials (Blakeman and Prioleau) each measured the pressure again.
- They ran out of time, and couldn't measure the pressure of all of the footballs.

## The measurements

In [ ]:
#: all of the measurements

footballs = pd.read_csv('./deflategate.csv')
footballs.head()

## Combining the measurements

- Both officials measured each ball.
- Their measurements are slightly different.
- We average them to get a combined weight.

In [ ]:
footballs['Combined'] = (footballs.Blakeman + footballs.Prioleau) / 2
footballs.head()

## Differences in average pressure

- At first glance, it looks as though the Patriots footballs are at a lower pressure.

In [ ]:
#:: group means
footballs.groupby('Team').mean()

- We could do a permutation test for difference in mean pressure.
- But that wouldn't point towards cheating.
    - The Patriot's balls *started* at a lower psi.
- The allegations were that the Patriots *deflated* the balls.
    - We want to check to see if the Patriots balls lost more pressure than the Colts'.

## Calculating the pressure drop

- We therefore calculate the drop in pressure for each ball.
- Patriots' started at 12.5 psi, Colts' started at 13.
- We make an array with starting pressure for each ball.
- Handy function: `np.where(array_of_true_and_false, true_value, false_value)`.
    - Replaces `True` in the array with `true_value`, and `False` with `false_value`.

In [ ]:
#...starting_pressure
starting_pressure = np.where(
    footballs['Team'] == 'Patriots',
    12.5,
    13
)
starting_pressure

## Calculating the pressure drop

In [ ]:
#: add the drop to the table
footballs['Pressure Drop'] = starting_pressure - footballs['Combined']

footballs.head()

## The question

- Did the Patriots' footballs drop in pressure more than the Colts'?
- A/B test!
- **Null hypothesis**: The drop in pressures for both teams came from the same distribution.
    - By chance, the Patriots' footballs deflated more.
- **Alternative hypothesis**: No, the Patriots' footballs deflated more than one would expect due to random chance alone.

## Permutation test

- We run a permutation test to see if this is a significant difference.
- Use the statistic 'Pressure Drop'
- Permute the drop in pressure (or the team column), many times.

In [ ]:
observed_difference, differences = run_perm_test_mean(footballs, 'Team', 'Pressure Drop')

## Conclusion

In [ ]:
#: visualize
pd.Series(differences).plot(kind='hist', density=True, alpha=0.8)
plt.scatter(observed_difference, 0, color='red', s=40);

- It doesn't look good for the Patriots. What is the p-value?

In [ ]:
# p-value
np.count_nonzero(differences >= observed_difference) / len(differences)

In [ ]:
#: p-value
np.count_nonzero(differences >= observed_difference) / n_repetitions

## Caution!

- We conclude that it is unlikely that the difference in mean pressure drop is due to chance alone.
- But this doesn't establish *causation*.
- That is, did the Patriots *deliberately* deflate their footballs?
- This was an *observational* study; to establish causation, we need an RCT (Randomized Controlled Trial).